In [10]:
import pickle as pk
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
# file_path = 'datasets/file_1.xlsx'  
# df1 = pd.read_excel(file_path, usecols=[2, 3, 4, 5], header=None, skiprows=2)
# medians = df1.median()
# absolute_differences = df1.subtract(medians).abs()
# mean_absolute_differences = absolute_differences.mean()
# print(mean_absolute_differences.shape)
# mean_absolute_differences = mean_absolute_differences.values
# print(mean_absolute_differences)
# d_mad = {}
# for env in range(len(mean_absolute_differences)):
#     d_mad[env] = mean_absolute_differences[env]
##
## change
file_path = 'datasets/file_1.xlsx'
df1 = pd.read_excel(file_path, usecols=[2, 3, 4, 5], header=None, skiprows=2)

d_mad = {env: {} for env in range(4)}
for env in range(4):
    env_data = df1.iloc[:, env]
    for trial_no in range(10):
        y_train, y_test = train_test_split(env_data, test_size=0.33, random_state=trial_no)
        y_train_median = y_train.median()
        mad = abs(y_test - y_train_median).mean()
#         print(y_train.shape, y_test.shape, y_train_median, y_test.median(), mad)        
        d_mad[env][trial_no] = mad

In [12]:
num_trials = 10 ## should be 10
list_env = [0,1,2,3] ## should be [0,1,2,3,4,5]

In [13]:
d_result = {}
list_d_result = []
for trial_no in range(num_trials):
    for env_no in list_env: 
        with open('datasets/' + 'd_result_' + str(trial_no) + '_' + str(env_no) + '.pickle', 'rb') as handle:
            d = pk.load(handle)
            list_d_result.append(d)
            d_result.update(d)

In [14]:
# d_result.keys()

In [15]:
d2 = {}
for env in range(4):
    if env not in d2.keys():
        d2[env] = {}
    for model in ['ridge', 'elastic', 'krr', 'svr']:
        if model not in d2[env].keys():
            d2[env][model] = {}
        for emb in ['regular', 'PCA', 'Siamese']:
            if emb not in d2[env][model].keys():
                d2[env][model][emb] = []
            for trial in range(10):
                mae = d_result[(str(trial), str(env), emb, model)]                
                # value = 1-(mae/d_mad[env])
                # change
                value = 1-(mae/d_mad[env][trial])
                d2[env][model][emb].append(value)

In [16]:
d3 = {}
for env in range(4):
    if env not in d3.keys():
        d3[env] = {}
    for model in ['ridge', 'elastic', 'krr', 'svr']:
        if model not in d3[env].keys():
            d3[env][model] = {}
        for emb in ['regular', 'PCA', 'Siamese']:
            if emb not in d3[env][model].keys():
                d3[env][model][emb] = ''
            d3[env][model][emb] = f'{np.mean(d2[env][model][emb]):.2f} ± {np.std(d2[env][model][emb]):.2f}'

In [17]:
rows = []
for env in range(4):
    for model in ['ridge', 'elastic', 'krr', 'svr']:
        row = [env, model]  # Start each row with env and model
        for emb in ['regular', 'PCA', 'Siamese']:
            row.append(d3[env][model][emb])  # Append the value from the dictionary
        rows.append(row)

column_names = ['env', 'model', 'regular', 'PCA', 'Siamese']
df = pd.DataFrame(rows, columns=column_names)
# df.drop(['env', 'model'], axis=1, inplace=True)
# csv_file_path = 'datasets/data_d2.csv'  # Replace with your desired file path
# xlsx_file_path = 'datasets/data_d2.xlsx'
# df.to_csv(csv_file_path, index=False)
# df.to_excel(xlsx_file_path, index=False)
## change
csv_file_path = 'datasets/data_d2b.csv'  # Replace with your desired file path
xlsx_file_path = 'datasets/data_d2b.xlsx'
df.to_csv(csv_file_path, index=False)
df.to_excel(xlsx_file_path, index=False)

In [18]:
## 19
##
## 0 - PBE-D3
## 1 - BEEF-vdW
## 2 - RPBE
## 3 - scan+rVV10
## 
## Functional Specific Model (1 to 1) x 768-chEMBL Fingerprint

In [18]:
df

,env,model,regular,PCA,Siamese
0,0,ridge,-0.28 ± 0.22,-0.15 ± 0.16,-0.00 ± 0.11
1,0,elastic,-0.02 ± 0.13,-0.06 ± 0.14,-0.00 ± 0.12
2,0,krr,0.14 ± 0.14,0.10 ± 0.06,0.02 ± 0.09
3,0,svr,0.12 ± 0.08,0.09 ± 0.07,-0.00 ± 0.09
4,1,ridge,-0.28 ± 0.20,-0.12 ± 0.14,-0.03 ± 0.12
5,1,elastic,-0.03 ± 0.15,0.00 ± 0.03,-0.02 ± 0.12
6,1,krr,-0.03 ± 0.21,0.02 ± 0.12,-0.04 ± 0.13
7,1,svr,0.02 ± 0.11,-0.04 ± 0.11,-0.04 ± 0.14
8,2,ridge,-0.21 ± 0.20,-0.03 ± 0.10,-0.00 ± 0.12
9,2,elastic,0.01 ± 0.15,0.02 ± 0.05,0.01 ± 0.12
